In [ ]:
import numpy as np
import casadi as ca
import gym
import gym_custom_envs
import time
import random
# import scipy.linalg as alg
from gym import wrappers

In [ ]:
from IPython.display import clear_output
env = gym.make("Quadrotor2D-v0")

In [ ]:
m = env.m
I = env.I
gravity = env.gravity
r = env.l/2
l = env.l
T = 5.
N = (1/env.dt).__int__()
q = ca.MX.sym("q", 3, 1)
dq = ca.MX.sym("dq", 3, 1)
u = ca.MX.sym("u", 2, 1)
state = ca.vertcat(q, dq)
L = ca.sumsqr(q) + ca.sumsqr(dq) + ca.sumsqr(u)

x_ddot = -((u[0, 0] + u[1, 0])*ca.sin(q[2, 0]))/m
y_ddot = (((u[0, 0] + u[1, 0])*ca.cos(q[2, 0])) - (m*gravity))/m
theta_ddot = ((u[0, 0] - u[1, 0])*l/2)/I

ddq = ca.vertcat(x_ddot, y_ddot, theta_ddot)
state_dot = ca.vertcat(dq, ddq)

print(q, dq, u, L)
print(state_dot)

In [ ]:
# Fixed step Runge-Kutta 4 integrator
M = 4 # RK4 steps per interval
DT = T/N/M
f = ca.Function('f', [state, u], [state_dot, L])
X0 = ca.MX.sym('X0', 6, 1)
U = ca.MX.sym('U', 2, 1)
X = X0
Q = 0
for j in range(M):
   k1, k1_q = f(X, U)
   k2, k2_q = f(X + DT/2 * k1, U)
   k3, k3_q = f(X + DT/2 * k2, U)
   k4, k4_q = f(X + DT * k3, U)
   X=X+DT/6*(k1 +2*k2 +2*k3 +k4)
   Q = Q + DT/6*(k1_q + 2*k2_q + 2*k3_q + k4_q)
F = ca.Function('F', [X0, U], [X, Q],['x0','p'],['xf','qf'])


In [ ]:
# Evaluate at a test point
Fk = F(x0=[0.2,0.3, 0.2,0.3, 0.2,0.3],p=[0.2,0.3])
print(Fk['xf'])
print(Fk['qf'])

In [ ]:
# Formulate the NLP
def fNLP(s):
    # Start with an empty NLP
    w=[]
    w0 = []
    lbw = []
    ubw = []
    J = 0
    g=[]
    lbg = []
    ubg = []
    print(s)
    Xk = ca.MX(s)
    print(m , g)
    for k in range(N):
        # New NLP variable for the control
#         if k == N - 1:
#             Xk = ca.MX([0, 0, 0, 0 , 0, 0])
            
        Uk = ca.MX.sym('U_' + str(k), 2, 1)
        w += [Uk]
        lbw += [-m*gravity, -m*gravity]
        ubw += [m*gravity, m*gravity]
        w0 += [0, 0]

        # Integrate till the end of the interval
        Fk = F(x0=Xk, p=Uk)
        Xk = Fk['xf']
        J=J+Fk['qf']
        
                # Add inequality constraint
        if k == N - 1:
            g += [Xk]
            lbg += [-.01]*6
            ubg += [.01]*6

    return w,w0,lbw,ubw,J,g,lbg,ubg 


In [ ]:
for i in range(10):
    s = env.reset().reshape(1, 6)[0]
    w,w0,lbw,ubw,J,g,lbg,ubg = fNLP(s)
    # Create an NLP solver
    prob = {'f': J, 'x': ca.vertcat(*w), 'g': ca.vertcat(*g)}
    solver = ca.nlpsol('solver', 'ipopt', prob)

    # Solve the NLP
    sol = solver(x0=w0, lbx=lbw, ubx=ubw, lbg=lbg, ubg=ubg)
    w_opt = sol['x']

    # Plot the solution
    u_opt = w_opt
    print(u_opt)
    x_opt = [s]
    for k in range(N):
        Fk = F(x0=x_opt[-1], p=ca.DM([u_opt[k], u_opt[k+1]]))
        x_opt += [Fk['xf'].full()]
#     x1_opt = [r[0] for r in x_opt]
#     x2_opt = [r[1] for r in x_opt]

#     tgrid = [T/N*k for k in range(N+1)]
    done = False
    jj = 0
    while not done:
#         K, S = lqr(A, B, Q, R)
        
        a = [u_opt[jj].__float__(),u_opt[jj+1].__float__()] 
        print(a)
#         print('Theta = ', round(np.rad2deg(s[0, 2]), 3))
#         print('u = ', a.T)
#         print("Reaction = ", round((a[0] + a[1])[0, 0] - m*g, 3))
#         print("Cost = ", round(np.dot(s, np.dot(S, s.T))[0, 0], 3))

        ns, r, done, _ = env.step([a[0], a[1]])
        s = ns.reshape(1, 6)
        jj += 1
        env.render()
        
        clear_output(wait=True)
        time.sleep(0.008)
env.close()